# AllSisters Scraping

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from random import randint
import time, urllib, json, unicodedata

In [2]:
# Headers to imitate browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Mobile Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.5',
    'DNT': '1',
    'Referer': 'https://www.google.co.uk',
    'Upgrade-Insecure-Requests': '1'
}

In [3]:
main_URL = 'https://allsisters.com'
URL_extension_for_products = '/products.json?limit=250&page='
current_page_number = 1


all_URLs_and_JSON = {}
current_product_list = None


while current_product_list != []:
    # make the full json url
    url = main_URL + URL_extension_for_products + str(current_page_number)
    print(url)
    
    # Get the page
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as request:
        s = request.read()
        response = json.loads(s)
        current_product_list = response['products']
    
    time.sleep(randint(4,6))
    
    # construct the URL
    for product in current_product_list:
        product_url = main_URL + '/products/' + product['handle']
        all_URLs_and_JSON[product_url] = product
        
    current_page_number += 1

https://allsisters.com/products.json?limit=250&page=1
https://allsisters.com/products.json?limit=250&page=2


In [4]:
print(len(all_URLs_and_JSON))
print(all_URLs_and_JSON)

135
{'https://allsisters.com/products/cesto-1-palma': {'id': 6547872612449, 'title': 'TRAMUNTANA', 'handle': 'cesto-1-palma', 'body_html': '<p>The palm heart handbag is completely hand-braided, its elaboration is very laborious and its process requires a lot of time. 100% ecological. No two bags are the same.\xa0<meta charset="utf-8">Handcrafted bags and hats made from palm, each one is one of a kind.<br data-mce-fragment="1"></p>\n<p><span>Measurements: 22 cm and 38 cm diameter approx.</span></p>\n<p><span>All PALMA hats and bags could be worn and also decorate your home.</span><br class=""></p>\n<p><br></p>', 'published_at': '2021-03-24T17:41:41+01:00', 'created_at': '2021-03-17T18:41:25+01:00', 'updated_at': '2021-03-24T17:41:41+01:00', 'vendor': 'allSisters', 'product_type': '', 'tags': [], 'variants': [{'id': 39274091151457, 'title': 'Default Title', 'option1': 'Default Title', 'option2': None, 'option3': None, 'sku': '', 'requires_shipping': True, 'taxable': True, 'featured_image

In [5]:
def find_product_name(soup, product_JSON):
    
    product_title = product_JSON['title'].strip()
    
    return product_title

In [22]:
def find_material(soup, product_JSON):
    materials = {}
    
    materials_container = soup.find('div', id = 'desc').find_all('p')
    
    materials_container = [item.text.strip() for item in materials_container if item.text.strip() != '']
    
    materials_list = []
    
    for item in materials_container:
        print(item)
        if item[0].isdigit():
            materials_list = item.split('-')

    materials_list = [item.strip().replace('%', '% &ND') for item in materials_list]

    materials_split = [item.split('&ND') for item in materials_list]

    for item in materials_split:
        materials[item[1].strip().lower()] = item[0].strip()
            
    return materials

In [18]:
def find_color(soup, product_JSON, the_description):
    
    product_title = product_JSON['title'].strip().lower()
    print(product_title)
    
    if 'black' in product_title:
        color = ['black']
        
    elif 'white' in product_title:
        color = ['white']
        
    elif 'colors' in product_title:
        color = ['multi-colored']
        
    elif len(product_JSON['options']) == 2:
        if product_JSON['options'][1]['name'] == 'Color':   
            color_container = product_JSON['options'][1]

            if color_container['name'] == 'Color':
                color = color_container['values']
                color = [item.lower() for item in color]
    elif 'black' and 'white' in the_description[0].lower():
        color = ['black and white']
    elif 'black' in the_description[0].lower():
        color = ['black']
    elif 'white' in the_description[0].lower():
        color = ['white']
    else: 
        color = None
        
    return color

In [8]:
def find_price(soup, product_JSON):

    price_text = soup.find('h2', id = 'price-preview').text.strip().split(' ')
    
    price = (price_text[0], price_text[1])
    
    return price

In [9]:
def find_image(soup, product_JSON):
    
    image_urls_container = product_JSON['images']
    
    image_urls = [item['src'] for item in image_urls_container]

    return image_urls

In [10]:
def find_brand(soup, product_JSON):
    
    brand = product_JSON['vendor']
    
    return brand

In [11]:
def find_description(soup, product_JSON):
    
    description_container = soup.find('div', id = 'desc').find_all('p')
    
    description = []
    
    for item in description_container:
        if item.text[0].isdigit():
            break
        else:
            description_section = item.text.strip()
            if description_section != '':
                description.append(unicodedata.normalize("NFKD", description_section))
    
    return description

In [12]:
def find_category(soup, product_JSON):
    
    category = [product_JSON['product_type'].strip().lower()]
    
    return category

In [19]:
def scrape_single_product(URL, html_soup, product_JSON):
    product = {}
    
    the_description = find_description(html_soup, product_JSON)
    
    print('got name')
    product['Name'] = find_product_name(html_soup, product_JSON)
    
    print('getting material')
    product['Material'] = find_material(html_soup, product_JSON)
    
    print('getting color')
    product['Color'] = find_color(html_soup, product_JSON, the_description)
        
    print('getting price')
    product['Price'] = find_price(html_soup, product_JSON)
    
    print('getting URL')
    product['URL'] = URL

    print('getting image')
    product['Image'] = find_image(html_soup, product_JSON)
    
    print('getting brand name')
    product['Brand_name'] = find_brand(html_soup, product_JSON)
    
    print('getting description')
    product['Description'] = the_description
    
    print('getting category')
    product['Category'] = find_category(html_soup, product_JSON)
    
    return product

In [14]:
def is_available(product_JSON):
    variants = product_JSON['variants']
    
    is_available = False
    
    for variant in variants:
        availability = variant['available']
        if availability == True:
            is_available = True
    
    return is_available

In [15]:
# These 3 cells scrape a single product
URL_to_scrape = 'https://allsisters.com/products/serena-leggins'
page = requests.get(URL_to_scrape, headers = headers)
single_soup = bs(page.content, 'html.parser')

In [16]:
product_as_json = None
URL_to_scrape_JSON = URL_to_scrape + ".json"

single_req = urllib.request.Request(URL_to_scrape_JSON, headers=headers)
with urllib.request.urlopen(single_req) as request:
    s = request.read()
    response = json.loads(s)
    product_as_json = response['product']
    print(product_as_json)

{'id': 4562483576929, 'title': 'SERENA LEGGINGS', 'body_html': '<p><span>Loads of design and big doses of green energy are present in these mid-rise leggings made with recycled and ecological fabrics. Black tights with </span><span>white block details in strategic areas: on the sides to sharpen your waist, </span><span>and on the ankles to stylize your figure. A very versatile piece designed to make you feel comfortable and beautiful while pampering your body. </span><span>Seam details in the knees and calves add an extra dose of support (and fantasy). One, two, three. Ready, steady and…go!</span></p>\n<p>\xa0<span>All designs are made by hand in Portugal and in Recycled Polyester, with GRS\xa0<strong>(Global\xa0</strong></span><span><strong>Recycle Standard)</strong>\xa0and\xa0<strong>OEKO-Tex Standard\xa0100\xa0certificate.</strong></span></p>\n<p><span><strong><img height="59" width="166" alt="" src="https://cdn.shopify.com/s/files/1/0827/0589/files/OE-100_1_480x480.jpg?v=1606385721

In [17]:
singleProductResult  = [scrape_single_product(URL_to_scrape, single_soup, product_as_json)]

print(singleProductResult)

product_data_frame = pd.DataFrame(data = singleProductResult,)

product_data_frame.to_csv('single_item.csv', encoding='utf-8-sig')

pd.DataFrame(data = singleProductResult)

got name
getting material
Loads of design and big doses of green energy are present in these mid-rise leggings made with recycled and ecological fabrics. Black tights with white block details in strategic areas: on the sides to sharpen your waist, and on the ankles to stylize your figure. A very versatile piece designed to make you feel comfortable and beautiful while pampering your body. Seam details in the knees and calves add an extra dose of support (and fantasy). One, two, three. Ready, steady and…go!
All designs are made by hand in Portugal and in Recycled Polyester, with GRS (Global Recycle Standard) and OEKO-Tex Standard 100 certificate.
Sustainable techno-fabric made with Econyl® - 100% regenerated polyamide fiber from post-consumer materials.
78% recycled polyamide - 22% elastan
*We recommend to wash by hand in cold water with an appropriate gentle cleaner. No bleach. Lay flat in shade to dry.*
*No returns or exchanges are allowed in our special sales (up to 30%). Please, con

,Name,Material,Color,Price,Brand_name,Description,Category
0,SERENA LEGGINGS,"{'recycled polyamide': '78%', 'elastan': '22%'}",[black and white],"(EUR, 77.00)",allSisters,[Loads of design and big doses of green energy...,[sport]


In [23]:
all_products = []
failed_urls = []
unavailable_products = []

# counter = 0

for url, the_JSON in all_URLs_and_JSON.items():
    try:

        print(url)
        
        if is_available(the_JSON):
            print(is_available(the_JSON))
            page = requests.get(url, headers = headers)
            soup = bs(page.content, 'html.parser')

            product = scrape_single_product(url, soup, the_JSON)
            print(product)
            if product['Material']:
                all_products.append(product)
            
        else:
            unavailable_products.append(url)

        time.sleep(randint(2,5))
  
    except:
        failed_urls.append(url)
    
#     if counter == 20:
#         break
#     counter += 1

print('\nSummary\n')
print(len(failed_urls), 'Failed URLs: ', failed_urls)
print(len(unavailable_products), 'Unavailable Products: ', unavailable_products)
print('Total products collected: ', len(all_products))
print('Done!')

https://allsisters.com/products/cesto-1-palma
True
https://allsisters.com/products/sombrero-2-palma
True
got name
getting material
Authentic traditional palm leaf hat from Mallorca. Handcrafted bags and hats made from palm, each one is one of a kind.
Measurements: 37 cm. diameter approx.
getting color
soller
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name': 'SOLLER', 'Material': {}, 'Color': None, 'Price': ('EUR', '30.00'), 'URL': 'https://allsisters.com/products/sombrero-2-palma', 'Image': ['https://cdn.shopify.com/s/files/1/0827/0589/products/OLDSISTERSESTUDIO20211254.jpg?v=1616072125', 'https://cdn.shopify.com/s/files/1/0827/0589/products/OLDSISTERSESTUDIO20211257.jpg?v=1616072125', 'https://cdn.shopify.com/s/files/1/0827/0589/products/OLDSISTERSESTUDIO20211870_V2.jpg?v=1616072125', 'https://cdn.shopify.com/s/files/1/0827/0589/products/OLDSISTERSESTUDIO20211877_V2.jpg?v=1616072126', 'https://cdn.shopify.com/s/files/1/0827/0589/p

https://allsisters.com/products/elixir-luna
True
got name
getting material
Elixir Luna is a 100% organic body oil, handmade and suitable for all skin types.
Thanks to its active principles, this elixir provides a great antioxidant regenerative power and deep hydration. Emotionally it helps to vibrate in love, peace and goodness, transforms emotions and enhances femininity and sensuality.
*For returns and exchanges please contact at: info@seaskinlife.com
getting color
elixir luna
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name': 'Elixir Luna', 'Material': {}, 'Color': None, 'Price': ('EUR', '78.00'), 'URL': 'https://allsisters.com/products/elixir-luna', 'Image': ['https://cdn.shopify.com/s/files/1/0827/0589/products/aceite_luna_2_1024x1024_d46f6e6d-a6c2-4834-b27e-e99daf3c55de.jpg?v=1616672223', 'https://cdn.shopify.com/s/files/1/0827/0589/products/aceite_luna.jpg?v=1616672223'], 'Brand_name': 'SeaSkin', 'Description': ['Elixir Luna 

https://allsisters.com/products/nimu-aegeria-bathrobe-stone
True
got name
getting material
Aegeria bathrobe comes with a versatile and practical hood. It is made in waffle weave, pure organic cotton. The unusual combination of a three-dimensional weave with a hypoallergenic and natural fiber like cotton, increases the absorbency of the fabric, while decreasing the drying time. The result is a very light, but very resistant bathrobe.
This bathrobe is made of a particular texture that allows for a light and pleasant scrub giving a wellbeing feeling on the skin.
MADE IN ITALY
100% COTTON
*For returns and exchanges please contact at: cs@nimuroma.com
getting color
nimu | aegeria bathrobe stone
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name': 'Nimu | Aegeria Bathrobe Stone', 'Material': {'cotton': '100%'}, 'Color': None, 'Price': ('EUR', '200.00'), 'URL': 'https://allsisters.com/products/nimu-aegeria-bathrobe-stone', 'Image': ['https://

got name
getting material
Woven in the blue geometric Ischia pattern, inspired by the Italian island in the 50s and the metaesquemas artworks from Brazilian artist Helio Oiticica, this luxurious Linen/Cotton Towel is crafted from soft linen and cotton-jacquard, and is both soft and resistant to the sand and sea water. It is quick-drying at the same time. The Towel can be used both at the beach or for a picnic in the park. It can also be used as a sarong or as a shawl.
MADE IN PORTUGAL
27% LINEN 73% COTTON
210 GR/M2
BLUE COLOR PRINT. HEM DETAIL AND LOGO LABEL
SIZE 170X135 CM
*For returns and exchanges please contact at: cs@nimuroma.com
https://allsisters.com/products/they-breathe-print
True
got name
getting material
EARTH SERIES COLLECTION by DAVID LOPEZ
Location : Redwoods, CA, Ca. All photographs are printed on smooth matte Fine Art Paper, 100% cotton, acids free, designed for premium quality prints.
+ If you are after a different size, please contact directly at info@along.fm with yo

https://allsisters.com/products/copia-de-olympic-top-bathing-suit-black
True
got name
getting material
Sporty halter top with contrasting geometric pattern on mock-neck featuring textured print inspired by the art work of Sol Lewitt. The extra coverage in front is contrasted by the cut-out triangle in the back. An athletic option for a good splash! Available in all black, or black or white with a contrasting neck pattern.
All designs are made by hand in Portugal and in Recycled Polyester, with GRS (Global Recycle Standard) and OEKO-Tex Standard 100 certificate.
Sustainable techno-fabric made with Econyl® - 100% regenerated polyamide fiber from post-consumer materials.
78% recycled polyamide - 22% elastan
*No returns or exchanges are allowed in our special sales (up to 30%). Please, contact us for assistance to carefully select your size.
getting color
olympic top bathing suit in color
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name

...

https://allsisters.com/products/persei-bottom-bathing-suit-black
True
got name
getting material
Go sexy with this cute little bottom, ideal for the sporty allSisters girl that likes to take risks.
All designs are made by hand in Portugal and in Recycled Polyester, with GRS (Global Recycle Standard) and OEKO-Tex Standard 100 certificate.
Sustainable techno-fabric made with ECONYL® - 100% regenerated polyamide fiber from pre- and post-consumer waste.
78% recycled polyamide - 22% elastan
*No returns or exchanges are allowed in our special sales (up to 30%). Please, contact us for assistance to carefully select your size.
getting color
persei bottom bathing suit black
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name': 'PERSEI Bottom Bathing Suit Black', 'Material': {'recycled polyamide': '78%', 'elastan': '22%'}, 'Color': ['black'], 'Price': ('EUR', '40.00'), 'URL': 'https://allsisters.com/products/persei-bottom-bathing-suit-black', 'Im

https://allsisters.com/products/orionis-bottom-bathing-suit
True
got name
getting material
A stylish black and white bottom with ivory details that provides an exceptional balance of comfort and performance. The high-waisted design offers ample coverage at the back. Looking chic on the beach was never so easy!
All designs are made by hand in Portugal and in Recycled Polyester, with GRS (Global Recycle Standard) and OEKO-Tex Standard 100 certificate.
Sustainable techno-fabric made with ECONYL® - 100% regenerated polyamide fiber from pre- and post-consumer waste.
78% recycled polyamide - 22% elastan
*We recommend to wash by hand in cold water with an appropriate gentle cleaner. No bleach. Lay flat in shade to dry.*
*No returns or exchanges are allowed in our special sales (up to 30%). Please, contact us for assistance to carefully select your size.
getting color
orionis bottom bathing suit
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'N

https://allsisters.com/products/scorpii-bottom-bathing-suit-white
True
got name
getting material
Ultra comfortable high-waisted briefs that can be easily combined with the SCORPII or the allSisters CARINAE top. The perfect choice for your mix and match wardrobe!
All designs are made by hand in Portugal and in Recycled Polyester, with GRS (Global Recycle Standard) and OEKO-Tex Standard 100 certificate.
Sustainable techno-fabric made with ECONYL® - 100% regenerated polyamide fiber from pre- and post-consumer waste.
78% recycled polyamide - 22% elastan
*No returns or exchanges are allowed in our special sales (up to 30%). Please, contact us for assistance to carefully select your size.
getting color
sc0orpii bottom bathing suit white
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name': 'SC0ORPII Bottom Bathing Suit White', 'Material': {'recycled polyamide': '78%', 'elastan': '22%'}, 'Color': ['white'], 'Price': ('EUR', '45.00'), 'URL': '

https://allsisters.com/products/carinae-top-bathing-suit-black
True
got name
getting material
The quintessential bikini set, with a sensual and athletic feel that makes it simply irresistible! An ultra chic top that offers maximum support and comfort. Its seductive neckline and asymmetric back will make you steal the spotlight!
All designs are made by hand in Portugal and in Recycled Polyester, with GRS (Global Recycle Standard) and OEKO-Tex Standard 100 certificate.
Sustainable techno-fabric made with ECONYL® - 100% regenerated polyamide fiber from pre- and post-consumer waste.
78% recycled polyamide - 22% elastan
*No returns or exchanges are allowed in our special sales (up to 30%). Please, contact us for assistance to carefully select your size.
getting color
carinae top bathing suit black
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name': 'CARINAE Top Bathing Suit Black', 'Material': {'recycled polyamide': '78%', 'elastan': '22%

https://allsisters.com/products/cassiopea-swimsuit-white
True
got name
getting material
Get into a summer mood with our fabulous CASSIOPEA swimsuit, which will take you straight to the catwalk. An elegant one shoulder monokini that looks awesome in white, making the perfect complement to your tan, while in black will bring out the star in you. The choice is yours!
All designs are made by hand in Portugal and in Recycled Polyester, with GRS (Global Recycle Standard) and OEKO-Tex Standard 100 certificate.
Sustainable techno-fabric made with ECONYL® - 100% regenerated polyamide fiber from pre- and post-consumer waste.
78% recycled polyamide - 22% elastan
*No returns or exchanges are allowed in our special sales (up to 30%). Please, contact us for assistance to carefully select your size.
getting color
cassiopea swimsuit white
getting price
getting URL
getting image
getting brand name
getting description
getting category
{'Name': 'CASSIOPEA Swimsuit White', 'Material': {'recycled polyamide

In [19]:
# fail_again_URLs = []
# for url in failed_urls:
#     try:

#         req = urllib.request.Request(url, headers=headers)
        
#         with urllib.request.urlopen(req) as request:
#             page = request.read()
#             soup = bs(page, 'html.parser')
#             all_products.append(scrape_single_product(url, soup))
            
#             time.sleep(randint(5,15))
            
#     except:
#         fail_again_URLs.append(url)

In [24]:
product_data_frame = pd.DataFrame(data = all_products)

product_data_frame.to_csv('allsisters_table.csv', encoding='utf-8-sig')

pd.DataFrame(data = all_products)

,Name,Material,Color,Price,URL,Image,Brand_name,Description,Category
0,Nimu | Agrippa Kimono Bathrobe Shell,{'cotton': '100%'},None,"(EUR, 200.00)",https://allsisters.com/products/nimu-agrippa-k...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,NIMU,[Agrippa bathrobe comes in a modern and chic K...,[home]
1,Nimu | Agrippa Kimono Bathrobe Stone,{'cotton': '100%'},None,"(EUR, 200.00)",https://allsisters.com/products/nimu-agrippa-k...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,NIMU,[Agrippa bathrobe comes in a modern and chic K...,[home]
2,Nimu | Agrippa Kimono Bathrobe White,{'cotton': '100%'},[white],"(EUR, 200.00)",https://allsisters.com/products/nimu-agrippa-k...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,NIMU,[Agrippa bathrobe comes in a modern and chic K...,[home]
3,Nimu | Aegeria Bathrobe Shell,{'cotton': '100%'},None,"(EUR, 200.00)",https://allsisters.com/products/nimu-aegeria-b...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,NIMU,[Aegeria bathrobe comes with a versatile and p...,[home]
4,Nimu | Aegeria Bathrobe Stone,{'cotton': '100%'},None,"(EUR, 200.00)",https://allsisters.com/products/nimu-aegeria-b...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,NIMU,[Aegeria bathrobe comes with a versatile and p...,[home]
...,...,...,...,...,...,...,...,...,...
89,CRUCIS AUSTRALIS Swimsuit White,"{'recycled polyamide': '78%', 'elastan': '22%'}",[white],"(EUR, 122.00)",https://allsisters.com/products/crucis-austral...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,allSisters,[The versatile CRUCIS AUSTRALIS one-piece is t...,[swimsuit]
90,CRUCIS AUSTRALIS Swimsuit Black,"{'recycled polyamide': '78%', 'elastan': '22%'}",[black],"(EUR, 122.00)",https://allsisters.com/products/crucis-austral...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,allSisters,[The versatile CRUCIS AUSTRALIS one-piece is t...,[swimsuit]
91,ANDROMEDAE Swimsuit Black,"{'recycled polyamide': '78%', 'elastan': '22%'}",[black],"(EUR, 122.00)",https://allsisters.com/products/andromedae-swi...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,allSisters,[Want to feel sexy and confident at the same t...,[swimsuit]
92,CASSIOPEA Swimsuit White,"{'recycled polyamide': '78%', 'elastan': '22%'}",[white],"(EUR, 122.00)",https://allsisters.com/products/cassiopea-swim...,[https://cdn.shopify.com/s/files/1/0827/0589/p...,allSisters,[Get into a summer mood with our fabulous CASS...,[swimsuit]
